In [1]:
from pyspark.sql import SparkSession
from operator import add
spark = SparkSession.builder.appName("MapReduceEmployeeInfo").master("spark://spark-master:7077").getOrCreate() 
# A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster.
sc = spark.sparkContext
sc.setLogLevel('FATAL')
lines_rdd = spark.sparkContext.textFile("/home/jovyan/data/employee_info.csv") # create a RDD from a text file

21/10/22 08:04:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html

How many police officers are there at the POLICE department?

In [2]:
def filterFun(x):
    if "POLICE OFFICER" in x: 
        return True
    return False

print(lines_rdd.filter(filterFun).count())

10639


Who is the lieutenant with the highest salary at the FIRE department?

In [30]:
from operator import add
import re
def filterFun2(x):
    if "FIRE" in x and re.search(r'\b,LIEUTENANT,\b', x) is not None:  
        return True
    return False

def mapFun(x):
    vals = x.split(',')   
    return (float(vals[-2]),vals[1] + ' ' + vals[0])

fire_lie_rdd = lines_rdd.filter(filterFun2)
output = fire_lie_rdd.map(mapFun).sortByKey(ascending=False).take(1) 
salary, name = output[0]
print(name)
print(salary)

TIM P EDWARDS
114846.0


In [31]:
# Stop the spark context
spark.stop()